Idea: We can leverage automated fact-checking frameworks for bias detection.

https://direct.mit.edu/tacl/article/doi/10.1162/tacl_a_00454/109469/A-Survey-on-Automated-Fact-Checking

This article highlights the following pipeline:

![image.png](attachment:image.png)

For our purposes, we will use a slightly modified version of this:
1) Claim detection: Transformer classifier
2) Source checking: Which sources are saying similar things? If we know the source, we can also reference MediaBias chart for their standings.
3) Bias detection: Transformer classifier
4) Prediction & Check with LLM

# Step 1: Transformer classifier for claim detection.

There's 3 methods I wanna try (ranked from easiest to hardest):
- Feature extraction classifier
- Fine-tuning output layers
- Fine-tuning all layers

But before we do any of that, let's define a dataset for claim detection.

In [ ]:
!pip install -U datasets huggingface_hub fsspec lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.1/821.1 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 572.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# We will be using the LIAR2 dataset https://aclanthology.org/P17-2067/ because it contains relevant language to our purpose (article bias detection)

import pandas as pd
import datasets

dataset = datasets.load_dataset("chengxuphd/liar2")

statement_train, y_train = dataset["train"]["statement"], dataset["train"]["label"]
statement_val, y_val = dataset["validation"]["statement"], dataset["validation"]["label"]
statement_test, y_test = dataset["test"]["statement"], dataset["test"]["label"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/19.0M [00:00<?, ?B/s]

valid.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/18369 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2297 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2296 [00:00<?, ? examples/s]

The "label" provides information about how true a statement is, but has been proven inaccurate at least when I ran it (I got about 31-32% test accuracies). Instead, we're going to train a one-class classifier for this task

Now, let's work with the transformer using this example: https://github.com/rasbt/MachineLearning-QandAI-book/blob/main/supplementary/q18-using-llms/

1. Tokenization & Numericalization

In [ ]:
# Make sure all these are installed.
import os.path as op

import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

import numpy as np
import pandas as pd
import torch

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("Tokenizer input max length:", tokenizer.model_max_length)
print("Tokenizer vocabulary size:", tokenizer.vocab_size)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenizer input max length: 512
Tokenizer vocabulary size: 30522


In [ ]:
tokenized_train = tokenizer(list(statement_train), padding=True, truncation=True, return_tensors="pt")
tokenized_val = tokenizer(list(statement_val), padding=True, truncation=True, return_tensors="pt")
tokenized_test = tokenizer(list(statement_test), padding=True, truncation=True, return_tensors="pt")

2. DistilBERT as Feature Extraction

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("distilbert-base-uncased")
model.to(device);

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset

# Step 1 — create a raw Hugging Face dataset from your input text
raw_train = Dataset.from_list([{"statement": s} for s in statement_train])
raw_val = Dataset.from_list([{"statement": s} for s in statement_val])
raw_test = Dataset.from_list([{"statement": s} for s in statement_test])

# Step 2 — tokenize using map so the result stays a Dataset object
tokenized_train = raw_train.map(lambda x: tokenizer(x["statement"], padding=True, truncation=True), batched=True)
tokenized_val = raw_val.map(lambda x: tokenizer(x["statement"], padding=True, truncation=True), batched=True)
tokenized_test = raw_test.map(lambda x: tokenizer(x["statement"], padding=True, truncation=True), batched=True)

Map:   0%|          | 0/18369 [00:00<?, ? examples/s]

Map:   0%|          | 0/2297 [00:00<?, ? examples/s]

Map:   0%|          | 0/2296 [00:00<?, ? examples/s]

In [ ]:
import torch

@torch.inference_mode()
def get_output_embeddings(batch):
    input_ids = torch.tensor(batch["input_ids"]).to(device)
    attention_mask = torch.tensor(batch["attention_mask"]).to(device)

    output = model(input_ids, attention_mask=attention_mask).last_hidden_state[:, 0]
    return {"features": output.cpu().numpy()}

In [ ]:
import time
start = time.time()

train_features = tokenized_train.map(get_output_embeddings, batched=True, batch_size=10)

Map:   0%|          | 0/18369 [00:00<?, ? examples/s]

In [ ]:
import time
start = time.time()

val_features = tokenized_val.map(get_output_embeddings, batched=True, batch_size=10)

Map:   0%|          | 0/2297 [00:00<?, ? examples/s]

In [ ]:
import time
start = time.time()

test_features = tokenized_test.map(get_output_embeddings, batched=True, batch_size=10)

Map:   0%|          | 0/2296 [00:00<?, ? examples/s]

In [ ]:
# Uncomment as needed!
import numpy as np

# Saving
# np.save("X_train.npy", np.stack(train_features["features"]))
# np.save("X_val.npy", np.stack(val_features["features"]))
# np.save("X_test.npy", np.stack(test_features["features"]))

# Loading
# X_train = np.load("X_train.npy")
# X_val = np.load("X_val.npy")
# X_test = np.load("X_test.npy")
# y_train = np.array(y_train)
# y_val = np.array(y_val)
# y_test = np.array(y_test)

In [ ]:
X_train = np.array(train_features["features"])
y_train = np.array(y_train)

X_val = np.array(val_features["features"])
y_val = np.array(y_val)

X_test = np.array(test_features["features"])
y_test = np.array(y_test)

3) Train a one-class model

In [ ]:
from sklearn.svm import OneClassSVM

# Fit only on "claim" features
model = OneClassSVM(kernel='rbf', gamma='scale', nu=0.05)
model.fit(X_train)

# Predict on validation and test sets
val_preds = model.predict(X_val)
test_preds = model.predict(X_test)

In [ ]:
# Interpret output: +1 means in-class (claim), -1 means outlier (not claim)
correct = 0
incorrect = 0
total = 0
for val in val_preds:
  if val == 1:
    correct += 1
  else:
    incorrect += 1
  total += 1
print(f"Accuracy: {correct/total}")

Accuracy: 0.9442751414888986


Claim detection model:

In [ ]:
# OneClassSVM initializer.
import numpy as np

X_train = np.load("X_train.npy")
X_val = np.load("X_val.npy")
X_test = np.load("X_test.npy")

from sklearn.svm import OneClassSVM

# Fit only on "claim" features
svm = OneClassSVM(kernel='rbf', gamma='scale', nu=0.05)
svm.fit(X_train)

# Predict on validation and test sets
val_preds = svm.predict(X_val)
test_preds = svm.predict(X_test)

def quick_accuracy(predictions):
  """
  Assumptions
  - 1 is correct (in-class), -1 is incorrect (out-class)
  """
  correct = 0
  incorrect = 0
  total = 0
  for prediction in predictions:
    if prediction == 1:
      correct += 1
    else:
      incorrect += 1
    total += 1
  print(f"Accuracy: {correct/total}")

quick_accuracy(val_preds)
quick_accuracy(test_preds)

import pickle
with open('one_class_svm.pkl', 'wb') as f:
    pickle.dump(svm, f)


Accuracy: 0.9442751414888986
Accuracy: 0.9499128919860628


In [ ]:
# Pipeline for new texts

# I'm honestly not sure if we need these libraries
import os.path as op
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from sklearn.feature_extraction.text import CountVectorizer

# Extra libraries we actually need
import numpy as np
import pandas as pd

# Initialize PyTorch
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("Tokenizer input max length:", tokenizer.model_max_length)
print("Tokenizer vocabulary size:", tokenizer.vocab_size)

from transformers import AutoModel
model = AutoModel.from_pretrained("distilbert-base-uncased")
model.to(device);

# Split the article into sentences, and tokenize each sentence.
import nltk
nltk.download('punkt_tab')
article = "It was only when the administration felt Newsom was not restoring order in the city — and after Trump watched the situation escalate for 24 hours and White House officials saw imagery of federal law enforcement officers with lacerations and other injuries — that the president moved to deploy the Guard, according to the official, who was granted anonymity to discuss private deliberations.   The president has put hundreds of National Guard troops on the streets to quell protests over his administration’s immigration raids, a deployment that state and city officials say has only inflamed tensions. “I understand the importance of deporting criminal aliens, but what we are witnessing are arbitrary measures to hunt down people who are complying with their immigration hearings — in many cases, with credible fear of persecution claims — all driven by a Miller-like desire to satisfy a self-fabricated deportation goal,” said Garcia, referring to Stephen Miller, a White House deputy chief of staff and key architect of Trump’s immigration crackdown.   Trump’s speedy deployment in California of troops against those whom the president has alluded to as “insurrectionists” on social media is a sharp contrast to his decision to issue no order or formal request for National Guard troops during the insurrection at the U.S. Capitol on Jan. 6, 2021, despite his repeated and false assertions that he had made such an offer.   The president and his top immigration aides accused the governor of mismanaging the protests, with border czar Tom Homan asserting in a Fox News interview Monday that Newsom stoked anti-ICE sentiments and waited two days to declare unlawful assembly in the city."
sentences = nltk.sent_tokenize(article)
from datasets import Dataset
raw_sentences = Dataset.from_list([{"text": s} for s in sentences])
tokenized_sentences = raw_sentences.map(lambda x: tokenizer(x["text"], padding=True, truncation=True), batched=True)

# Get embeddings
@torch.inference_mode()
def get_output_embeddings(batch):
    input_ids = torch.tensor(batch["input_ids"]).to(device)
    attention_mask = torch.tensor(batch["attention_mask"]).to(device)

    output = model(input_ids, attention_mask=attention_mask).last_hidden_state[:, 0]
    return {"features": output.cpu().numpy()}
import time
start = time.time()
sentence_features = tokenized_sentences.map(get_output_embeddings, batched=True, batch_size=10)

# Predict
predictions = svm.predict(sentence_features["features"])

# Format for output
for prediction, sentence in zip(predictions, sentences):
  label = "claim" if prediction == 1 else "not claim"
  print(f"{label}: {sentence}")




cpu
Tokenizer input max length: 512
Tokenizer vocabulary size: 30522


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

claim: It was only when the administration felt Newsom was not restoring order in the city — and after Trump watched the situation escalate for 24 hours and White House officials saw imagery of federal law enforcement officers with lacerations and other injuries — that the president moved to deploy the Guard, according to the official, who was granted anonymity to discuss private deliberations.
claim: The president has put hundreds of National Guard troops on the streets to quell protests over his administration’s immigration raids, a deployment that state and city officials say has only inflamed tensions.
claim: “I understand the importance of deporting criminal aliens, but what we are witnessing are arbitrary measures to hunt down people who are complying with their immigration hearings — in many cases, with credible fear of persecution claims — all driven by a Miller-like desire to satisfy a self-fabricated deportation goal,” said Garcia, referring to Stephen Miller, a White House d

Claim detection final product:

In [ ]:
class ClaimDetector():
  def __init__(self, pkl_path: str):

    # Initialize the model itself
    import pickle
    with open(pkl_path, 'rb') as f:
      self.svm = pickle.load(f)

    # I'm honestly not sure if we need these libraries
    import os.path as op
    import lightning as L
    from lightning.pytorch.loggers import CSVLogger
    from lightning.pytorch.callbacks import ModelCheckpoint
    from sklearn.feature_extraction.text import CountVectorizer

    # Extra libraries we actually need
    import numpy as np
    import pandas as pd
    from datasets import Dataset
    from datasets.utils.logging import disable_progress_bar

    # Initialize PyTorch
    import torch
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    from transformers import AutoTokenizer
    self.tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    print("Tokenizer input max length:", tokenizer.model_max_length)
    print("Tokenizer vocabulary size:", tokenizer.vocab_size)
    from transformers import AutoModel
    self.model = AutoModel.from_pretrained("distilbert-base-uncased")
    self.model.to(device);

    # NLTK
    import nltk
    nltk.download('punkt_tab')

    print("Model successfully loaded!")

  def predict(self, article: str, print_output=True):

    # Split the article into sentences, then tokenize the sentences.
    sentences = nltk.sent_tokenize(article)
    raw_sentences = Dataset.from_list([{"text": s} for s in sentences])
    tokenized_sentences = raw_sentences.map(lambda x: self.tokenizer(x["text"], padding=True, truncation=True), batched=True)

    # Embedding function
    @torch.inference_mode()
    def get_output_embeddings(batch):
        input_ids = torch.tensor(batch["input_ids"]).to(device)
        attention_mask = torch.tensor(batch["attention_mask"]).to(device)

        output = model(input_ids, attention_mask=attention_mask).last_hidden_state[:, 0]
        return {"features": output.cpu().numpy()}
    sentence_features = tokenized_sentences.map(get_output_embeddings, batched=True, batch_size=10)

    # Predict & get estimated probabilities
    X = sentence_features["features"]
    predictions = svm.predict(X)
    from scipy.special import expit
    prob_estimates = expit(svm.decision_function(X))

    # Format for output
    output = []
    for estimated_probability, prediction, sentence in zip(prob_estimates, predictions, sentences):
      label = "claim" if prediction == 1 else "not claim"
      if print_output:
        #print(f"{int(estimated_probability * 10**5) / 10**5:.5f} {label}: {sentence}")
        print(f"{estimated_probability} {label}: {sentence}")
      output.append((estimated_probability, label, sentence))
    return output


Example usage:

In [ ]:
# Initialize
pkl_path = "one_class_svm.pkl"
claim_detector = ClaimDetector(pkl_path)

# Works on articles and single sentences.
article = "It was only when the administration felt Newsom was not restoring order in the city — and after Trump watched the situation escalate for 24 hours and White House officials saw imagery of federal law enforcement officers with lacerations and other injuries — that the president moved to deploy the Guard, according to the official, who was granted anonymity to discuss private deliberations.   The president has put hundreds of National Guard troops on the streets to quell protests over his administration’s immigration raids, a deployment that state and city officials say has only inflamed tensions. “I understand the importance of deporting criminal aliens, but what we are witnessing are arbitrary measures to hunt down people who are complying with their immigration hearings — in many cases, with credible fear of persecution claims — all driven by a Miller-like desire to satisfy a self-fabricated deportation goal,” said Garcia, referring to Stephen Miller, a White House deputy chief of staff and key architect of Trump’s immigration crackdown.   Trump’s speedy deployment in California of troops against those whom the president has alluded to as “insurrectionists” on social media is a sharp contrast to his decision to issue no order or formal request for National Guard troops during the insurrection at the U.S. Capitol on Jan. 6, 2021, despite his repeated and false assertions that he had made such an offer.   The president and his top immigration aides accused the governor of mismanaging the protests, with border czar Tom Homan asserting in a Fox News interview Monday that Newsom stoked anti-ICE sentiments and waited two days to declare unlawful assembly in the city."
output = claim_detector.predict(article)
sentence = "Written by: Chengyi Li"
output = claim_detector.predict(sentence)
sentence = "Trump has been a good president"
output = claim_detector.predict(sentence)
sentence = "1 + 2 = 3"
output = claim_detector.predict(sentence)transformer

cpu
Tokenizer input max length: 512
Tokenizer vocabulary size: 30522
Model successfully loaded!


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

0.9999999999999971 claim: It was only when the administration felt Newsom was not restoring order in the city — and after Trump watched the situation escalate for 24 hours and White House officials saw imagery of federal law enforcement officers with lacerations and other injuries — that the president moved to deploy the Guard, according to the official, who was granted anonymity to discuss private deliberations.
1.0 claim: The president has put hundreds of National Guard troops on the streets to quell protests over his administration’s immigration raids, a deployment that state and city officials say has only inflamed tensions.
1.0 claim: “I understand the importance of deporting criminal aliens, but what we are witnessing are arbitrary measures to hunt down people who are complying with their immigration hearings — in many cases, with credible fear of persecution claims — all driven by a Miller-like desire to satisfy a self-fabricated deportation goal,” said Garcia, referring to Step

...

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

0.9999999999773486 claim: Trump has been a good president


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

1.1575716945089978e-22 not claim: 1 + 2 = 3


After testing this model out with more outputs (not shown), here are my findings:

Strengths:
- The model seems to be able to have a consistent definition for a class -- it's not just a random guess.
- To me, it seems that the model learned the grammatical features of what makes a political claim, however, some sentences outside the realm of politics also exhibit these features.
- The model is doing slightly better than chance at identify political texts rather than just any other texts.

Weaknesses:
- Almost all of the AP News article was classified as a claim
- Some false positives are seen in non-politically charged texts


Features I want this model to have:
- Ability to quantify degree of opinion (i.e. not classify everything in an article as a claim, even though most sentences in articles are indeed claims).
- Ability to distinguish between politically-charged articles vs. just texts.

# Step 2) Source checking

Based off of this annotated dataset, we're able to see which sources are left/right leaning at the time of the dataset being collected.

In [ ]:
!git clone https://github.com/ramybaly/Article-Bias-Prediction

Cloning into 'Article-Bias-Prediction'...
remote: Enumerating objects: 37585, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 37585 (delta 0), reused 0 (delta 0), pack-reused 37581 (from 1)
Receiving objects: 100% (37585/37585), 127.14 MiB | 9.23 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Updating files: 100% (37563/37563), done.


In [ ]:
# Load dataset
import pandas as pd
import numpy as np
import os
import json

# Initialize paths
repo_file = "Article-Bias-Prediction"
data_folder = f"{repo_file}/data"
json_folder = f"{data_folder}/jsons"
splits_folder = f"{data_folder}/splits"
media_splits_folder = f"{splits_folder}/media"
media_splits = [media_splits_folder + "/" + name for name in os.listdir(media_splits_folder)]

# Read the labels
articles_df = pd.concat([pd.read_csv(media_split, delimiter="\t") for media_split in media_splits])
columns = ['topic', 'source', 'bias', 'url', 'title', 'date', 'authors', 'content', 'content_original', 'source_url', 'bias_text', 'id']
for column in columns:
  articles_df[column] = ['' for _ in range(len(articles_df))]

# Append json info
for index, row in articles_df.iterrows():
  id = row['ID']
  filename = json_folder + "/" + id + ".json"
  with open(filename) as f:
    article = json.load(f)
    for column in article:
      articles_df.loc[articles_df['ID'] == id, column] = article[column]

In [ ]:
articles_df.to_csv('article-bias-df.csv', index=False)

In [ ]:
# Initialize Pytorch and relevant libraries again.
!pip install lightning

# I'm honestly not sure if we need these libraries
import os.path as op
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from sklearn.feature_extraction.text import CountVectorizer

# Extra libraries we actually need
import numpy as np
import pandas as pd
from datasets import Dataset
from datasets.utils.logging import set_verbosity_error, disable_progress_bar


# Initialize PyTorch
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("Tokenizer input max length:", tokenizer.model_max_length)
print("Tokenizer vocabulary size:", tokenizer.vocab_size)
from transformers import AutoModel
model = AutoModel.from_pretrained("distilbert-base-uncased")
model.to(device);

# NLTK
import nltk
nltk.download('punkt_tab')

# Embedding function
@torch.inference_mode()
def get_output_embeddings(batch):
    input_ids = torch.tensor(batch["input_ids"]).to(device)
    attention_mask = torch.tensor(batch["attention_mask"]).to(device)
    output = model(input_ids, attention_mask=attention_mask).last_hidden_state[:, 0]
    return {"features": output.cpu().numpy()}

def get_embedding(article: str):

    # Split the article into sentences, then tokenize the sentences.
    sentences = nltk.sent_tokenize(article)
    raw_sentences = Dataset.from_list([{"text": s} for s in sentences])
    tokenized_sentences = raw_sentences.map(lambda x: tokenizer(x["text"], padding=True, truncation=True), batched=True)
    sentence_features = tokenized_sentences.map(get_output_embeddings, batched=True, batch_size=10)

    # Output features
    return sentence_features

cuda:0


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Tokenizer input max length: 512
Tokenizer vocabulary size: 30522


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
import pandas as pd

# Load dataset
article_bias_df = pd.read_csv('article-bias-df.csv')

In [ ]:
from datasets import Dataset

# Let's focus on the articles as the unit for now.
raw_articles = Dataset.from_list([{"article": a} for a in article_bias_df['content']])
tokenized_articles = raw_articles.map(lambda x: tokenizer(x["article"], padding=True, truncation=True), batched=True)
articles_features = tokenized_articles.map(get_output_embeddings, batched=True, batch_size=10)
np.save("X_articles.npy", np.stack(articles_features["features"]))

# Save
from google.colab import files
files.download("X_articles.npy")

# Load
X = np.load("X_articles.npy")
y = np.array(article_bias_df['bias'])


Map:   0%|          | 0/30246 [00:00<?, ? examples/s]

Map:   0%|          | 0/30246 [00:00<?, ? examples/s]

In [ ]:
from datasets import Dataset
import nltk
nltk.download('punkt_tab')

# Let's convert this dataset into a sentence-based dataset
article_bias_df.drop(columns=["content_original", "id"], inplace=True)

# Tokenize sentences
sentence_level_data = []
for _, row in article_bias_df.iterrows():
    sentences = nltk.sent_tokenize(row["content"])
    for sent in sentences:
        new_row = row.to_dict()
        new_row["sentence"] = sent  # Replace article with sentence
        sentence_level_data.append(new_row)
sentence_bias_df = pd.DataFrame(sentence_level_data)

# Drop the original article content
sentence_bias_df.drop(columns=["content"], inplace=True)

# Save
sentence_bias_df.to_csv("sentence-bias-df.csv")


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,ID,bias,topic,source,url,title,date,authors,source_url,bias_text,sentence
0,sJfVGcfW6vWo04q7,2,media_bias,Reason,https://reason.com/archives/2017/03/15/ignoran...,Has the Ignorant Media Gotten Worse?,2017-03-15,"John Stossel, Peter Suderman, Noah Shepardson,...",www.reason.com,right,Has the media gotten worse ?
1,sJfVGcfW6vWo04q7,2,media_bias,Reason,https://reason.com/archives/2017/03/15/ignoran...,Has the Ignorant Media Gotten Worse?,2017-03-15,"John Stossel, Peter Suderman, Noah Shepardson,...",www.reason.com,right,Or am I just grouchier ?
2,sJfVGcfW6vWo04q7,2,media_bias,Reason,https://reason.com/archives/2017/03/15/ignoran...,Has the Ignorant Media Gotten Worse?,2017-03-15,"John Stossel, Peter Suderman, Noah Shepardson,...",www.reason.com,right,Every day I see things that are wrong or that ...
3,sJfVGcfW6vWo04q7,2,media_bias,Reason,https://reason.com/archives/2017/03/15/ignoran...,Has the Ignorant Media Gotten Worse?,2017-03-15,"John Stossel, Peter Suderman, Noah Shepardson,...",www.reason.com,right,As this week 's storm approached the East Coas...
4,sJfVGcfW6vWo04q7,2,media_bias,Reason,https://reason.com/archives/2017/03/15/ignoran...,Has the Ignorant Media Gotten Worse?,2017-03-15,"John Stossel, Peter Suderman, Noah Shepardson,...",www.reason.com,right,Here I blame my beloved free market : Predicti...
...,...,...,...,...,...,...,...,...,...,...,...
1291553,RKCKIDG8BJwhrmlK,0,middle_east,New York Times - News,http://www.nytimes.com/2014/08/14/world/middle...,"Militants’ Siege on Mountain in Iraq Is Over, ...",2014-08-14,"Helene Cooper, Michael D. Shear",www.nytimes.com,left,But the more viable route — going through Syri...
1291554,RKCKIDG8BJwhrmlK,0,middle_east,New York Times - News,http://www.nytimes.com/2014/08/14/world/middle...,"Militants’ Siege on Mountain in Iraq Is Over, ...",2014-08-14,"Helene Cooper, Michael D. Shear",www.nytimes.com,left,That option required the use of American airst...
1291555,RKCKIDG8BJwhrmlK,0,middle_east,New York Times - News,http://www.nytimes.com/2014/08/14/world/middle...,"Militants’ Siege on Mountain in Iraq Is Over, ...",2014-08-14,"Helene Cooper, Michael D. Shear",www.nytimes.com,left,The United States is at odds with President Ba...
1291556,RKCKIDG8BJwhrmlK,0,middle_east,New York Times - News,http://www.nytimes.com/2014/08/14/world/middle...,"Militants’ Siege on Mountain in Iraq Is Over, ...",2014-08-14,"Helene Cooper, Michael D. Shear",www.nytimes.com,left,The speed with which the Obama administration ...


In [ ]:
import pandas as pd
from datasets import Dataset

# Load
sentence_bias_df = pd.read_csv("sentence-bias-df.csv")

# Now let's focus on sentences.
raw_sentences = Dataset.from_list([{"sentence": s} for s in sentence_bias_df['sentence']])
tokenized_sentences = raw_sentences.map(lambda x: tokenizer(x["sentence"], padding=True, truncation=True), batched=True)
sentences_features = tokenized_sentences.map(get_output_embeddings, batched=True, batch_size=10) # TODO: Google Colab runs out of memory with this 1,291,558 length dataset. Need to find a work around.
np.save("X_sentences.npy", np.stack(sentences_features["features"]))

# Save
from google.colab import files
files.download("X_sentences.npy")

Map:   0%|          | 0/1291558 [00:00<?, ? examples/s]

Map:   0%|          | 0/1291558 [00:00<?, ? examples/s]

In [ ]:
# Load
X = np.load("X_sentences.npy")
y = np.array(sentence_bias_df['bias'])

Train some baseline models

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
# Try out some baseline models
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import pickle

# Define baseline models
models = {
    "Dummy (Most Frequent)": DummyClassifier(strategy="most_frequent"),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Naive Bayes": GaussianNB(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC(),
}

# Train and evaluate
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred, average="weighted"),
        "Precision": precision_score(y_test, y_pred, average="weighted"),
        "Recall": recall_score(y_test, y_pred, average="weighted"),
    })
    with open(f'{name[:3]}.pkl', 'wb') as f:
      pickle.dump(model, f)

# Display results
import pandas as pd
results_df = pd.DataFrame(results).sort_values(by="F1 Score", ascending=False)
print(results_df)

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                    Model  Accuracy  F1 Score  Precision    Recall
1     Logistic Regression  0.649879  0.649411   0.649482  0.649879
6  Support Vector Machine  0.603923  0.602426   0.603949  0.603923
5           Random Forest  0.545515  0.542536   0.547518  0.545515
3     K-Nearest Neighbors  0.523804  0.523408   0.526522  0.523804
2             Naive Bayes  0.459114  0.459761   0.463998  0.459114
4           Decision Tree  0.425832  0.426063   0.426515  0.425832
0   Dummy (Most Frequent)  0.361472  0.191943   0.130662  0.361472


Yikes a 54% accuracy. It's okay though we still have a few more things that we expect to improve accuracy:
- Training on claim detected sentences.
- Fine tuning last layers of transformer
- Fine tuning the entire transformer

With using the full article, we get 64% accuracy on Log Reg and SVM.

In [ ]:
class ArticleBiasDetector():
  def __init__(pkl_path: str):
    load